In [1]:
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
databaseName='messages'

# Use a service account
cred = credentials.Certificate('service_file.json')
firebase_admin.initialize_app(cred)
db = firestore.client()
collection_ref = db.collection(databaseName)
docs = collection_ref.stream()



In [2]:
# store all docs ids, except the test one
ids_array=[]
for doc in docs:
    if (doc.id !='D***************p'):
        ids_array.append(doc.id)


# store all messages in conversations
conversations_list=[]
for id in ids_array:
    collection_chat = db.collection(databaseName).document(id).collection('chat')
    docs=collection_chat.stream()
    document=[]
    for doc in docs:
        document.append(doc.to_dict())
    conversations_list.append(document)


conversations_list.sort(
    key=lambda conversation: 0 if not conversation else max(
        conversation,
        key= lambda m: m['createdAt']
    )['createdAt']
)

In [ ]:
trigger_list=["téléphone","telephone","tel","tél","direct","directe","directement", "appli", "application", "num","numéro","numero","numéro.", "numero.","06", "plateforme", "lydia", "paypal", "€"]

id_list=[]

if(trigger_list):
    for one_conversation in conversations_list:
        one_conversation.sort(key=lambda m: m['createdAt'])
        for one_message in one_conversation:
                if(any(word in one_message['text'].split() for word in trigger_list)):
                print("****From:", int(one_message['user']['_id']), 'at ', datetime.datetime.fromtimestamp(one_message['createdAt']/1000))
                for element in one_message['text'].split():
                    if(element in trigger_list):
                        print("\033[41;37m "+element+" \033[m",end=' ')
                    else:
                        print(element,end=' ')
                if not one_message['user']['_id'] in id_list :
                    id_list.append(int(one_message['user']['_id']) )
                if one_message.get('image'):
                    print("@@image@@: ", one_message['image'])
                print('\n')
    print( "("+str(id_list).strip('[]')+")" )
else:
    for one_conversation in conversations_list:
        one_conversation.sort(key=lambda m: m['createdAt'])
        for one_message in one_conversation:
            print("****From:", one_message['user']['_id'], 'at ', datetime.datetime.fromtimestamp(one_message['createdAt']/1000))
            print(one_message['text'])
            print(one_message)
            if one_message.get('image'):
                print("@@image@@: ", one_message['image'])
            print("")
